# First Simple Context Manager 

In [1]:
class LoggingContextManager:
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        return

The above context manager is simple, but not very functional.

In [2]:
with LoggingContextManager() as x:
    print(x)

In the with-statement `x` is bound to `LoggingContextManager`.  Now the `__enter__()` method of the context manager will be updated to return something else:

In [4]:
class LoggingContextManager:
    def __enter__(self):
        return "You are in a with-block!"

    def __exit__(self, exc_type, exc_val, exc_tb):
        return

In [5]:
with LoggingContextManager() as x:
    print(x)

You are in a with-block!


Now finish the context manager by having it log some text upon entry and exit:

In [6]:
class LoggingContextManager:
    def __enter__(self):
        print('LoggingContextManager.__enter__()')
        return "You are in a with-block!"

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('LoggingContextManager.__exit__({}, {}, {})'.format(exc_type, exc_val, exc_tb))
        return

In [7]:
with LoggingContextManager() as x:
    print(x)

LoggingContextManager.__enter__()
You are in a with-block!
LoggingContextManager.__exit__(None, None, None)


Now raise an exception from the with-block and see that the `__exit__()` method is still called, this time with exception information:

In [8]:
with LoggingContextManager() as x:
    raise ValueError("Something has gone wrong")
    print(x)

LoggingContextManager.__enter__()
LoggingContextManager.__exit__(<class 'ValueError'>, Something has gone wrong, <traceback object at 0x7fdbed34b408>)


ValueError: Something has gone wrong

Output from the `__enter__()` method is visible, also output from the `__exit_()` method indicates that an exception occured.

## `__enter__()`

The `__enter__()` method is called on the context-manager just before entering the with-block, and its return value is bound to the as-variable of the with-statement.

`__enter__()` is allowed to return anything it wants, including `None`, and the with-statement itself does not ever access or use this value.  It is very common, however, for context-managers to simply return themselves from `__enter__()`. Consider statements below:

In [9]:
with open('important_data.txt', 'wt') as f:
    f.write('The secret password is 12345')

In [10]:
with open('important_data.txt', 'r') as f:
    data = f.read()

In the above, `open()` returns a file object and `f` is expected to be bound to that file object, so the `__enter__()` method of the file must be returning the file object itself.  This can be verified by a simple experiment:

In [11]:
f = open('a_file', 'w')
with f as g:
    print(f is g)

True


In the above, first a file is opened without using a with-statement, binding the file to the name 'f'.  Then use 'f' as a context-manager in a with-statement, binding the result of `__enter__()` to the name `g` and see that `f` and `g` are the same object.

## `__exit__()`

Update `LoggingContextManger.__exit__()` to behave differently when an exception is raised:

In [4]:
class LoggingContextManager:
    def __enter__(self):
        print('LoggingContextManager.__enter__()')
        return "You are in a with-block!"

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            print('LoggingContextManager.__exit__: '
                  'normal exit detected')
        else:
            print('LoggingContextManager.__exit__: '
                  'Exception detected! '
                  'type={}, value={}, traceback={}'.format(exc_type, exc_val, exc_tb))
        return

The above revision of the implementation of `__exit__()` first checks whether `type` is `None`.  If it is, then this means that no exception was raised and a simple message is printed.

If `type` is no `None`, however, `__exit__()` prints a longer message that includes the exception information.

In [3]:
with LoggingContextManager():
    pass

LoggingContextManager.__enter__()
LoggingContextManager.__exit__ normal exit detected


The above output shows that normal exits are handled properly.  Now raise an exception from the with-block:

In [5]:
with LoggingContextManager():
    raise ValueError('Core meltdown imminent!')

LoggingContextManager.__enter__()
LoggingContextManager.__exit__: Exception detected! type=<class 'ValueError'>, value=Core meltdown imminent!, traceback=<traceback object at 0x7f8da744e7c8>


ValueError: Core meltdown imminent!

The above outpus shows that `LoggingContextManager` properly detects exceptions.